In [2]:
!pip install imblearn
!pip install xgboost
import librosa
import soundfile as sf
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.ensemble import AdaBoostRegressor
import pickle
from sklearn.utils import resample
!pip3 install numba==0.48.0


  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.8.1-py3-none-any.whl (189 kB)
  Using cached xgboost-1.5.1-py3-none-win_amd64.whl (106.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for numba: filename=numba-0.48.0-cp39-cp39-win_amd64.whl size=2040528 sha256=870421b23a323828238bcf7bbabb3de85778f5d44ca4c7686bf8efc1abfb1919
  Stored in directory: c:\users\abbas\appdata\local\pip\cache\wheels\67\f4\34\43855bda1e661dc1afedcb295d313a510239d901336fd636c3
  Running setup.py clean for llvmlite
Successfully built numba
Failed to build llvmlite
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.37.0


  ERROR: Command errored out with exit status 1:
   command: 'F:\Anaconda1\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\abbas\\AppData\\Local\\Temp\\pip-install-zhhr0ysp\\llvmlite_08206e3d226240d3813bbdc66109c8fc\\setup.py'"'"'; __file__='"'"'C:\\Users\\abbas\\AppData\\Local\\Temp\\pip-install-zhhr0ysp\\llvmlite_08206e3d226240d3813bbdc66109c8fc\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\abbas\AppData\Local\Temp\pip-wheel-cufmxtvj'
       cwd: C:\Users\abbas\AppData\Local\Temp\pip-install-zhhr0ysp\llvmlite_08206e3d226240d3813bbdc66109c8fc\
  Complete output (24 lines):
  running bdist_wheel
  F:\Anaconda1\python.exe C:\Users\abbas\AppData\Local\Temp\pip-install-zhhr0ysp\llvmlite_08206e3d226

In [3]:
mfccfeatures=[]
chromagramfeatures=[]
labels=[]
origin=r"E:\3rdSem\Prog AI\Project\Files"
for i in os.listdir(origin):
    a=(os.path.join(origin,i))
    for j in os.listdir(a):
        b=(os.path.join(a,j))
        for k in os.listdir(b):
            path=os.path.join(b,k)
            audio_path = path
            x , sr = librosa.load(audio_path)
            MFCC_Features = librosa.feature.mfcc(y=x, sr=sr)
            mfcc=MFCC_Features.flatten().tolist()
            hop_length = 512
            chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length)
            chr=chromagram.flatten().tolist()
            mfccfeatures.append(mfcc)
            chromagramfeatures.append(chr)
            labels.append(i)

    


In [4]:
def padding(x,y):
    size=[]
    for i in x:
        size.append(len(i))
    n=max(size)
    print(n)
    for i in x:
        while(len(i)<n):
            i.append(0)   
    return x      

In [5]:
mfcc=pd.DataFrame(padding(mfccfeatures,7600))
chroma=pd.DataFrame(padding(chromagramfeatures,5000))


7600
4560


In [6]:
df=pd.concat([mfcc,chroma],axis=1,ignore_index=True)
le=LabelEncoder()
a=le.fit_transform(labels)
y=pd.DataFrame(a)

In [7]:
df.to_csv('Data.csv')
y.to_csv("labels.csv")

In [17]:
df=pd.read_csv('Data.csv')
y=pd.read_csv("labels.csv")

In [18]:
sm=SMOTE()
print(len(mfccfeatures),len(y))
X_sm,y_sm=sm.fit_resample(df,y)


5919 5919


ValueError: y should be a 1d array, got an array of shape (5919, 2) instead.

In [ ]:
pickle.dump(sm, open("sm2.pkl", 'wb'))

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,test_size=0.1,random_state=22)
print("shape of X_train = ",X_train.shape)
print("shape of X_test = ",X_test.shape)
print("shape of y_train = ",y_train.shape)
print("shape of y_test = ",y_test.shape)

shape of X_train =  (11768, 12160)
shape of X_test =  (1308, 12160)
shape of y_train =  (11768, 1)
shape of y_test =  (1308, 1)


In [ ]:
knn=KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
knn.fit(X_train, y_train)
pred=knn.predict(X_test)
knn.score(X_test,y_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test, pred))
print("F1 Score : ",f1_score(y_test, pred, average='micro')*100)
pickle.dump(knn, open("knn2.pkl", 'wb'))

Accuracy :  73.01223241590215
Report : 
               precision    recall  f1-score   support

           0       0.77      0.96      0.86       334
           1       0.84      0.28      0.41       309
           2       0.69      0.95      0.80       328
           3       0.71      0.70      0.70       337

    accuracy                           0.73      1308
   macro avg       0.75      0.72      0.69      1308
weighted avg       0.75      0.73      0.70      1308

F1 Score :  73.01223241590215


In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
pred=dt.predict(X_test)
dt.score(X_test,y_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test, pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
pickle.dump(dt, open("dt2.pkl", 'wb'))

Accuracy :  68.42507645259938
Report : 
               precision    recall  f1-score   support

           0       0.80      0.85      0.82       334
           1       0.56      0.49      0.52       309
           2       0.73      0.76      0.75       328
           3       0.62      0.62      0.62       337

    accuracy                           0.68      1308
   macro avg       0.68      0.68      0.68      1308
weighted avg       0.68      0.68      0.68      1308

F1 Score :  67.8008676617675


In [23]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=4, random_state=0)
etc.fit(X_train, y_train)
pred = etc.predict(X_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test, pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
pickle.dump(etc, open("etc2.pkl", 'wb'))

Accuracy :  82.79816513761467
Report : 
               precision    recall  f1-score   support

           0       0.90      0.97      0.94       334
           1       0.73      0.57      0.64       309
           2       0.91      0.96      0.93       328
           3       0.74      0.80      0.77       337

    accuracy                           0.83      1308
   macro avg       0.82      0.82      0.82      1308
weighted avg       0.82      0.83      0.82      1308

F1 Score :  81.90191514440887


In [19]:
!pip install lightgbm
import lightgbm as lgb #https://www.kaggle.com/prashant111/lightgbm-classifier-in-python
lg = lgb.LGBMClassifier()
lg.fit(X_train, y_train)
pred=lg.predict(X_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test, pred))
print("F1 Score : ",f1_score(y_test, pred, average='macro')*100)
pickle.dump(lg, open("lg2.pkl", 'wb'))

Accuracy :  92.2782874617737
Report : 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       334
           1       0.80      0.93      0.86       309
           2       0.98      0.95      0.96       328
           3       0.93      0.83      0.88       337

    accuracy                           0.92      1308
   macro avg       0.93      0.92      0.92      1308
weighted avg       0.93      0.92      0.92      1308

F1 Score :  92.26400655486802


In [20]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier(n_estimators = 50)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print ("Accuracy : " , accuracy_score(y_test,pred)*100)  
print("Report : \n", classification_report(y_test, pred))
print("F1 Score : ",f1_score(y_test, pred, average='micro')*100)
pickle.dump(rfc, open("rfc2.pkl", 'wb'))

Accuracy :  90.06116207951071
Report : 
               precision    recall  f1-score   support

           0       0.94      0.99      0.96       334
           1       0.82      0.83      0.83       309
           2       0.96      0.98      0.97       328
           3       0.87      0.80      0.83       337

    accuracy                           0.90      1308
   macro avg       0.90      0.90      0.90      1308
weighted avg       0.90      0.90      0.90      1308

F1 Score :  90.06116207951071
